In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan
from ld_gan import visualize
import numpy as np
import torch
from time import time

In [20]:
X, Y = ld_gan.data_proc.data_loader.load_data(1, verbose=1, resize = 64)

100%|██████████| 102/102 [00:12<00:00,  8.07it/s]

load data from '/export/home/oblum/projects/ls_gan/data/flowers_102/jpg_128'


In [21]:
project, epoch = "xf_11111_mc.py", 650
enc = ld_gan.utils.model_handler.load_model(project, epoch, "enc")

loaded model 'projects/xf_11111_mc.py/model/e_0650.pth'


In [84]:
sampler = ld_gan.sample.nn_sampler_life(enc, X, Y, 256)

In [85]:
t = time()
out = sampler.next()
time() - t

0.7522940635681152